## Experiment 1

Simulation and visualization of superposition of a single qubit as noise patterns on a canvas of variable size.
This notebook is for further experimentation and adjustments within the scripts.

### Including all modules

In [1]:
# Qiskit for Quantum computation
from qiskit import QuantumCircuit, Aer, transpile, execute
from qiskit.providers.aer.noise import NoiseModel, QuantumError, ReadoutError, pauli_error

# PILlow for image generation
from PIL import Image, ImageColor

# Handy math libraries
import numpy as np
import math

# For saving files in current directory
import os

### Definition of all functions needed to execute the experiment

In [2]:
# Simulate Quantum computation

def run_qc(circuit, backend, output, n_shots):

    # -------------------------------------------------------------------------------
    
    # Build basic bit-flip error noise model

    # Example error probabilities
    p_reset = 0.03
    p_meas = 0.1
    p_gate1 = 0.05

    # QuantumError objects
    error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
    error_gate2 = error_gate1.tensor(error_gate1)

    # Add errors to noise model
    noise_bit_flip = NoiseModel()
    noise_bit_flip.add_all_qubit_quantum_error(error_reset, "reset")
    noise_bit_flip.add_all_qubit_quantum_error(error_meas, "measure")
    noise_bit_flip.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
    noise_bit_flip.add_all_qubit_quantum_error(error_gate2, ["cx"])
    
    # -------------------------------------------------------------------------------
    
    # Execution and options
    
    # Load simulator (Aer)
    backend_simulate = Aer.get_backend('aer_simulator')
    
    # Execute on Aer
    if (backend == 'sim'):
        run = execute(circuit,
                      backend_simulate,
                      shots = n_shots,
                      memory = True).result()
        if (output == 'count'):
            out = run.get_counts()
        if (output == 'memory'):
            out = run.get_memory()
    
    # Execute on Aer + noise model
    if (backend == 'sim_noise'):
        run = execute(circuit,
                      backend_simulate,
                      noise_model=noise_bit_flip,
                      shots = n_shots,
                      memory = True).result()
        if (output == 'count'):
            out = run.get_counts()
        if (output == 'memory'):
            out = run.get_memory()

    return out

In [3]:
# Generate EPISODIC image

def sample_noise_episodic(sidelength, path):
    data_out = []
    
    # Quantum circuit
    qc = QuantumCircuit(1)
    qc.h(0)
    qc.measure_all()

    # Measurements
    result = run_qc(qc, 'sim', 'memory', sidelength ** 2)
    result = np.reshape(result, (sidelength, sidelength)).astype(float)
    data_out.append(result)
    
    # Image assembling
    a = int(len(data_out) ** (1/2))
    b = int(len(data_out) ** (1/2))
    c = int(len(data_out[0]))
    d = int(len(data_out[0][0]))
    
    img = Image.new('RGB', (a * c, b * d))
    for i in range(a):
        for j in range(b):
            for k in range(c):
                for l in range(d):
                    color = int(data_out[j + (a * i)][k][l] * 255)
                    img.putpixel(((k + (j * c)), l + (i * d)), (color, color, color))
            
    #img.show()
    img.save(path)

In [4]:
# Generate CONTINUOUS image

def sample_noise_continous(sidelength, width, path):
    data_out = []
    
    # Quantum circuit
    for i in range(width):
        qc = QuantumCircuit(1)
        qc.h(0)
        qc.measure_all()

        # Measurements
        result = run_qc(qc, 'sim', 'memory', sidelength)
        data_out.append(result)
    
    # Image assembling
    img = Image.new('RGB', (width, sidelength))
    for j in range(sidelength):
        for k in range(width):
            color = int(int(data_out[k][j]) * 255)
            img.putpixel((k, j), (color, color, color))
    
    #img.show()
    img.save(path)

### Execution of functions with different iterative variations

In [105]:
# Exectution sample_noise_episodic

sidelength = 20

sample_noise_episodic(sidelength, f"episodic_{sidelength}x{sidelength}px.png")

In [6]:
#Execution sample_noise_episodic in a loop

# increasing sidelength 1*2, ..., 4*2 
for sidelength in range(1, 5):
    
    # repetition of execution = 4 images per sidelength
    for num_exe in range(4):
        sample_noise_episodic(2 * sidelength, f'episodic_{2 * sidelength}x{2 * sidelength}_{num_exe}.png')

In [108]:
# Execution sample_noise_continuous

width = 200
sidelength = 20

sample_noise_continous(sidelength, width, f'continous_{sidelength}x{width}px.png')